# Proprocessing dataset

In [2]:
import pandas as pd
import datasets
from datasets import load_dataset, Dataset, DatasetDict

Qdata_frame = pd.read_csv('../dataset/train3.csv')


c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# labelling
Qdata_frame['topic'] = Qdata_frame['topic'].apply(lambda x: x.strip().split(', '))
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
Qdf = Qdata_frame.join(pd.DataFrame(mlb.fit_transform(Qdata_frame['topic']), columns=mlb.classes_))

In [1]:
Qdf.to_csv(r'../dataset/train3_clean.csv')

NameError: name 'Qdf' is not defined

In [4]:
Qdata_Cleanframe = pd.read_csv('../dataset/train3_clean.csv')

In [5]:
max(Qdata_Cleanframe['String'].map(lambda x: len(x.split(' '))))

63

In [6]:
# split the train, test, and validation data
import sklearn
from sklearn.model_selection import train_test_split

Qdf_train, Qdf_remain = train_test_split(Qdata_Cleanframe, train_size=0.8)
Qdf_valid, Qdf_test = train_test_split(Qdf_remain, test_size=0.5)

In [7]:
len(Qdf_train.columns)

30

In [8]:
len(Qdf_valid.columns)

30

In [9]:
len(Qdf_test.columns)

30

In [10]:
# change to hugging face dataset type
trainDS = Dataset.from_pandas(Qdf_train)
validDS = Dataset.from_pandas(Qdf_valid)
testDS = Dataset.from_pandas(Qdf_test)
ds = DatasetDict()
ds["train"] = trainDS
ds["valid"] = validDS
ds["test"] = testDS
print(ds)

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'id', 'String', 'topic', 'type', 'Algorithm Design', 'Communication Software and Protocols', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Database', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Input and Output Devices', 'Intellectual Property', 'Internet Services and Applications', 'Introduction to HTML', 'Introduction to Information Processing', 'Multimedia Elements', 'Number System', 'Problem- Formulation and Analysis', 'Program Development', 'Program Testing and Debugging', 'Secondary Storage Devices', 'Spreadsheet', 'System Software', 'System Unit of a Computer System', 'The Networking and Internet Basics', 'Threats and Security on the Internet', 'Word Presentation', '__index_level_0__'],
        num_rows: 342
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'id', 'String', 'topic', 'type', 'Algorithm Design', 'Communication Software and Proto

In [11]:
#  clear not used columns
columns = ds.column_names
columns_toRemove = ["id", "type", "topic"]
new_ds = ds.remove_columns(columns_toRemove)
print(new_ds)


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'String', 'Algorithm Design', 'Communication Software and Protocols', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Database', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Input and Output Devices', 'Intellectual Property', 'Internet Services and Applications', 'Introduction to HTML', 'Introduction to Information Processing', 'Multimedia Elements', 'Number System', 'Problem- Formulation and Analysis', 'Program Development', 'Program Testing and Debugging', 'Secondary Storage Devices', 'Spreadsheet', 'System Software', 'System Unit of a Computer System', 'The Networking and Internet Basics', 'Threats and Security on the Internet', 'Word Presentation', '__index_level_0__'],
        num_rows: 342
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'String', 'Algorithm Design', 'Communication Software and Protocols', 'Computer System', 'Data Manipulation a

In [141]:
labels_original = [label for label in new_ds['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [13]:
# extract the labels
labels_original
id2label = {idx:label for idx, label in enumerate(labels_original)}
label2id = {label:idx for idx, label in enumerate(labels_original)}

In [143]:
Qdata_Cleanframe[labels_original].sum()

Algorithm Design                          48
Communication Software and Protocols      16
Computer System                            7
Data Manipulation and Analysis            11
Data Organisation and Data Control        16
Database                                  34
Elementary Web Authoring                   4
Health and Ethical Issues                 18
Input and Output Devices                  10
Intellectual Property                     18
Internet Services and Applications        19
Introduction to HTML                       4
Introduction to Information Processing    11
Multimedia Elements                       24
Number System                             31
Problem- Formulation and Analysis          4
Program Development                       38
Program Testing and Debugging              7
Secondary Storage Devices                 14
Spreadsheet                               30
System Software                           14
System Unit of a Computer System          35
The Networ

In [142]:
labels_original

['Algorithm Design',
 'Communication Software and Protocols',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Database',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Input and Output Devices',
 'Intellectual Property',
 'Internet Services and Applications',
 'Introduction to HTML',
 'Introduction to Information Processing',
 'Multimedia Elements',
 'Number System',
 'Problem- Formulation and Analysis',
 'Program Development',
 'Program Testing and Debugging',
 'Secondary Storage Devices',
 'Spreadsheet',
 'System Software',
 'System Unit of a Computer System',
 'The Networking and Internet Basics',
 'Threats and Security on the Internet',
 'Word Presentation']

In [15]:
from transformers import AutoTokenizer, BertForSequenceClassification
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [16]:
def encode_data(dataset):
    text = dataset["String"]
    # tokenize string
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=70)
    # create encoded array list with labels
    labelsBatch = {x: dataset[x] for x in dataset.keys() if x in labels}
    labels_matrix = np.zeros((len(text), len(labels)))
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labelsBatch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [17]:
encoded_ds = new_ds.map(encode_data, batched=True, remove_columns=new_ds['train'].column_names)
encoded_ds['train'][0]
encoded_ds['train'][0].keys()

100%|██████████| 1/1 [00:00<00:00, 250.47ba/s]


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [18]:
encoded_ds['train']['input_ids'][0]

[101,
 4480,
 2064,
 2338,
 2005,
 12436,
 14693,
 9323,
 2083,
 1037,
 4773,
 2609,
 1012,
 4480,
 2064,
 7276,
 1996,
 2335,
 10994,
 2005,
 1996,
 21725,
 2027,
 2215,
 2008,
 2145,
 2031,
 20563,
 2015,
 1012,
 2029,
 1997,
 1996,
 2206,
 11583,
 1997,
 3169,
 2190,
 6235,
 2023,
 2291,
 1029,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [19]:
[id2label[idx] for idx, label in enumerate(encoded_ds['train'][15]['labels']) if label == 1.0]

['Database', 'Spreadsheet']

# Model training

In [20]:
# TODO:  tokenize text, feature extraction, multilabel classification
import torch
from torch import cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

encoded_ds.set_format("torch")

Using device: cuda



In [34]:
from transformers import AutoModelForSequenceClassification, AutoModel
# define model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", problem_type="multi_label_classification", num_labels=len(labels), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [35]:
# train model
batch_size = 8
metric_name = "f1"

In [36]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"nbroad/ESG-BERT",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 30,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [38]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
    
def multi_label_metrics(predictions, labels, threshold=0.5):
    # apply sigmoid on predictions fitting (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [39]:
encoded_ds['train'][0]['labels'].type()

'torch.FloatTensor'

In [40]:
encoded_ds['train']['input_ids'][0]

tensor([  101,  4480,  2064,  2338,  2005, 12436, 14693,  9323,  2083,  1037,
         4773,  2609,  1012,  4480,  2064,  7276,  1996,  2335, 10994,  2005,
         1996, 21725,  2027,  2215,  2008,  2145,  2031, 20563,  2015,  1012,
         2029,  1997,  1996,  2206, 11583,  1997,  3169,  2190,  6235,  2023,
         2291,  1029,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [41]:
#forward pass
outputs = model(input_ids=encoded_ds['train']['input_ids'][0].unsqueeze(0), labels=encoded_ds['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.6724, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.3612,  0.2631,  0.4374, -0.0700,  0.7228,  0.1198, -0.3052, -0.1301,
          0.0266,  0.2273,  0.2286, -0.5257,  0.1255, -0.0676, -0.0962, -0.1973,
          0.0611, -0.4574, -0.5786, -0.2036, -0.1694,  0.2730, -0.1104,  0.2453,
         -0.1880]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [42]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [43]:
trainer.train()

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 342
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1290
  Number of trainable parameters = 109501465
  3%|▎         | 43/1290 [00:07<01:46, 11.69it/s] ***** Running Evaluation *****
  Num examples = 43
  Batch size = 8
                                                 
  3%|▎         | 43/1290 [00:07<01:46, 11.69it/s]Saving model checkpoint to nbroad/ESG-BERT\checkpoint-43
Configuration saved in nbroad/ESG-BERT\checkpoint-43\config.json


{'eval_loss': 0.34994110465049744, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.1363, 'eval_samples_per_second': 315.442, 'eval_steps_per_second': 44.015, 'epoch': 1.0}


Model weights saved in nbroad/ESG-BERT\checkpoint-43\pytorch_model.bin
tokenizer config file saved in nbroad/ESG-BERT\checkpoint-43\tokenizer_config.json
Special tokens file saved in nbroad/ESG-BERT\checkpoint-43\special_tokens_map.json
  7%|▋         | 85/1290 [00:12<01:42, 11.71it/s]***** Running Evaluation *****
  Num examples = 43
  Batch size = 8
                                                 
  7%|▋         | 86/1290 [00:12<01:42, 11.71it/s]Saving model checkpoint to nbroad/ESG-BERT\checkpoint-86
Configuration saved in nbroad/ESG-BERT\checkpoint-86\config.json


{'eval_loss': 0.24263913929462433, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_runtime': 0.1297, 'eval_samples_per_second': 331.573, 'eval_steps_per_second': 46.266, 'epoch': 2.0}


Model weights saved in nbroad/ESG-BERT\checkpoint-86\pytorch_model.bin
tokenizer config file saved in nbroad/ESG-BERT\checkpoint-86\tokenizer_config.json
Special tokens file saved in nbroad/ESG-BERT\checkpoint-86\special_tokens_map.json
 10%|█         | 129/1290 [00:18<01:37, 11.96it/s]***** Running Evaluation *****
  Num examples = 43
  Batch size = 8
                                                  
 10%|█         | 129/1290 [00:18<01:37, 11.96it/s]Saving model checkpoint to nbroad/ESG-BERT\checkpoint-129
Configuration saved in nbroad/ESG-BERT\checkpoint-129\config.json


...

Model weights saved in nbroad/ESG-BERT\checkpoint-1247\pytorch_model.bin
tokenizer config file saved in nbroad/ESG-BERT\checkpoint-1247\tokenizer_config.json
Special tokens file saved in nbroad/ESG-BERT\checkpoint-1247\special_tokens_map.json
100%|██████████| 1290/1290 [03:37<00:00, 11.85it/s]***** Running Evaluation *****
  Num examples = 43
  Batch size = 8
                                                   
100%|██████████| 1290/1290 [03:37<00:00, 11.85it/s]Saving model checkpoint to nbroad/ESG-BERT\checkpoint-1290
Configuration saved in nbroad/ESG-BERT\checkpoint-1290\config.json


{'eval_loss': 0.10597935318946838, 'eval_f1': 0.6410256410256411, 'eval_roc_auc': 0.7441214767156863, 'eval_accuracy': 0.4186046511627907, 'eval_runtime': 0.1258, 'eval_samples_per_second': 341.929, 'eval_steps_per_second': 47.711, 'epoch': 30.0}


Model weights saved in nbroad/ESG-BERT\checkpoint-1290\pytorch_model.bin
tokenizer config file saved in nbroad/ESG-BERT\checkpoint-1290\tokenizer_config.json
Special tokens file saved in nbroad/ESG-BERT\checkpoint-1290\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from nbroad/ESG-BERT\checkpoint-946 (score: 0.6410256410256411).
100%|██████████| 1290/1290 [03:39<00:00,  5.87it/s]

{'train_runtime': 219.8928, 'train_samples_per_second': 46.659, 'train_steps_per_second': 5.866, 'train_loss': 0.14222994516062182, 'epoch': 30.0}


TrainOutput(global_step=1290, training_loss=0.14222994516062182, metrics={'train_runtime': 219.8928, 'train_samples_per_second': 46.659, 'train_steps_per_second': 5.866, 'train_loss': 0.14222994516062182, 'epoch': 30.0})

In [44]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 43
  Batch size = 8
100%|██████████| 6/6 [00:00<00:00, 61.33it/s]


{'eval_loss': 0.11120515316724777,
 'eval_f1': 0.6410256410256411,
 'eval_roc_auc': 0.7441214767156863,
 'eval_accuracy': 0.4186046511627907,
 'eval_runtime': 0.2372,
 'eval_samples_per_second': 181.266,
 'eval_steps_per_second': 25.293,
 'epoch': 30.0}

In [100]:
# trainer.save_model("./temp_model")

Saving model checkpoint to ./temp_model
Configuration saved in ./temp_model\config.json
Model weights saved in ./temp_model\pytorch_model.bin
tokenizer config file saved in ./temp_model\tokenizer_config.json
Special tokens file saved in ./temp_model\special_tokens_map.json


In [45]:
encoded_ds["test"][0]

{'input_ids': tensor([ 101, 2029, 1997, 1996, 2206, 2024, 6413, 2043, 2478, 1037, 2591, 2865,
         4646, 1029,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [47]:
trainer.model.device


device(type='cuda', index=0)

In [85]:
test2 = encoded_ds['test']

In [86]:
test2 = test2.with_format("torch",device=device)


In [89]:
test2[0]

{'input_ids': tensor([ 101, 2029, 1997, 1996, 2206, 2024, 6413, 2043, 2478, 1037, 2591, 2865,
         4646, 1029,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        device='cuda:0'),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        device='cuda:0'),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 

In [ ]:
# from torch.utils.data import DataLoader
# test_loader = DataLoader(test2, batch_size=8)

# acc = []
# loop = tqdm(test_loader)
# for batch in loop:
#     with torch.ho_grad():
#         input_ids = batch['input_ids']
#         token_type_ids = batch['token_type_ids']
#         attention_mask = batch['attention_mask']
#         labels = batch['labels']

#         outputs = trainer.model(input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

#         loop.set_descriptions(f'Epoch {epoch}')
#         loop.set_postfix(loss = loss.item())



In [115]:
text = "When creating a database table in database software, what should users usually set?"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

In [116]:
logits = outputs.logits
logits.shape

torch.Size([1, 25])

In [117]:
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['Database']


In [123]:
predictions = trainer.predict(encoded_ds['test'])

***** Running Prediction *****
  Num examples = 43
  Batch size = 8
100%|██████████| 6/6 [00:00<00:00, 58.81it/s]


In [134]:
print(predictions.predictions)

[[-3.7618997 -2.9189916 -2.913126  ... -2.962478  -2.6815667 -3.3390715]
 [-2.7135108 -3.2827024 -3.6933084 ... -3.252916  -3.3481479 -2.7850587]
 [-0.7024888 -3.9971478 -3.7749178 ... -3.8887842 -3.465865  -3.984353 ]
 ...
 [-3.8829486 -3.4575887 -3.0203755 ... -3.0978239 -3.457069  -3.3682363]
 [ 1.6844727 -3.241483  -3.215388  ... -2.6664422 -2.3518958 -3.8822513]
 [-2.7291753 -3.064532  -3.40631   ... -3.595975  -3.2936034 -2.8933134]]


In [130]:

print(predictions.metrics)

{'test_loss': 0.11980658024549484, 'test_f1': 0.36065573770491804, 'test_roc_auc': 0.6176215827946085, 'test_accuracy': 0.16279069767441862, 'test_runtime': 0.2729, 'test_samples_per_second': 157.556, 'test_steps_per_second': 21.985}


In [135]:
model.save_pretrained("./Bert-model-v1")

Configuration saved in ./Bert-model-v1\config.json
Model weights saved in ./Bert-model-v1\pytorch_model.bin


In [137]:
trainer.evaluate(encoded_ds['test'])

***** Running Evaluation *****
  Num examples = 43
  Batch size = 8
100%|██████████| 6/6 [00:00<00:00, 52.42it/s]


{'eval_loss': 0.11980658024549484,
 'eval_f1': 0.36065573770491804,
 'eval_roc_auc': 0.6176215827946085,
 'eval_accuracy': 0.16279069767441862,
 'eval_runtime': 0.3234,
 'eval_samples_per_second': 132.956,
 'eval_steps_per_second': 18.552,
 'epoch': 30.0}

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Training model 2